### https://towardsdatascience.com/how-to-forecast-sales-with-python-using-sarima-model-ba600992fa7d

In [1]:
import warnings
import itertools
import numpy as np
from numpy import concatenate, savetxt, unique, array, subtract
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import pandas as pd
from pandas import merge, DataFrame, Series
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib
from math import sqrt
from statistics import mean


In [2]:
df = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True,
                 squeeze=True, dayfirst=True, engine='python')
df1a = pd.DataFrame(index =['Month'], columns=['prediction', 'actual'])
pred_date = '2019-03-01'

df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df.columns =['MeterNo', 'Month', 'kWh']
df_meters = pd.read_csv('CHL-Meters.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
df = merge(df, df_meters, on='MeterNo')
y = df.set_index('Month')
y.drop(y.columns[[0]], axis = 1, inplace = True)
y = y.sort_index()

ya = y.groupby(y.index)['kWh'].mean()
yc = y.groupby(y.index)['Area_m2'].mean()
z = merge(ya, yc, left_index=True, right_index=True)
print(z)


                   kWh    Area_m2
Month                            
2013-01-01  475.450704  72.517606
2013-02-01  468.250000  72.517606
2013-03-01  455.806338  72.517606
2013-04-01  390.042254  72.517606
2013-05-01  280.778169  72.517606
...                ...        ...
2020-10-01  343.144366  72.517606
2020-11-01  425.767606  72.517606
2020-12-01  554.538732  72.517606
2021-01-01  560.669014  72.517606
2021-02-01  515.232394  72.517606

[98 rows x 2 columns]


In [3]:
endog = z.iloc[:,0]
exog = z.iloc[:,1:]
mod = sm.tsa.statespace.SARIMAX(endog = endog,
                            order=(1, 1, 1),
                            seasonal_order=(0, 1, 1, 12),
                            exog = exog,
                            enforce_stationarity=False,
                            enforce_invertibility=False)
results = mod.fit()
pred = results.get_prediction(start=pd.to_datetime(pred_date), dynamic=False)
y_forecasted = pred.predicted_mean
y_forecasted =y_forecasted.to_frame()
y_truth = y[pred_date:]
df1 = merge(y_forecasted, y_truth, left_index=True, right_index=True)
df1.drop(df1.columns[[2]], axis = 1, inplace = True)
df1.columns =['prediction', 'actual']
df1['error'] = round(((df1['prediction'] - df1['actual'])),2)
print(df1)

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


            prediction  actual    error
2019-03-01  376.539023     586  -209.46
2019-03-01  376.539023     492  -115.46
2019-03-01  376.539023     630  -253.46
2019-03-01  376.539023     327    49.54
2019-03-01  376.539023    1156  -779.46
...                ...     ...      ...
2021-02-01  521.287353     234   287.29
2021-02-01  521.287353     486    35.29
2021-02-01  521.287353     513     8.29
2021-02-01  521.287353     430    91.29
2021-02-01  521.287353    1719 -1197.71

[6816 rows x 3 columns]


In [4]:
# Each meter per month
df2 = df1.dropna()
print(df2)

            prediction  actual    error
2019-03-01  376.539023     586  -209.46
2019-03-01  376.539023     492  -115.46
2019-03-01  376.539023     630  -253.46
2019-03-01  376.539023     327    49.54
2019-03-01  376.539023    1156  -779.46
...                ...     ...      ...
2021-02-01  521.287353     234   287.29
2021-02-01  521.287353     486    35.29
2021-02-01  521.287353     513     8.29
2021-02-01  521.287353     430    91.29
2021-02-01  521.287353    1719 -1197.71

[6816 rows x 3 columns]


In [5]:
mse = mean_squared_error(df2['actual'],df2['prediction'])
print(mse)

50508.65630251342


In [6]:
rmse = np.sqrt(mse)
print(rmse)

224.74130973746998


In [7]:
# Summary of each meter for each month
def Summary(x):
    return round(Series(index=['min','max', 'mean','sum','count'],data=[x.min(),x.max(),x.mean(),x.sum(),x.count()]),2)
df2.apply(Summary)

,prediction,actual,error
min,202.29,-1821.00,-1582.86
max,593.10,2152.00,2104.59
mean,365.35,368.06,-2.71
sum,2490201.94,2508683.00,-18477.28
count,6816.00,6816.00,6816.00


In [8]:
#Mean Square Error Calc
def mse(g):
    mse = mean_squared_error(g['actual'], g['prediction'])
    return Series({'mse':mse})

df2a = df2.groupby(df2.index)['prediction'].sum()
df2b = df2.groupby(df2.index)['actual'].sum()
df2c = df2.groupby(df2.index).apply(mse).reset_index()
df2c = df2c.set_index('index')
df3 = merge(df2a, df2b, left_index=True, right_index=True)
df3 = merge(df3, df2c, left_index=True, right_index=True)
df3['rmse'] = round(np.sqrt(df3['mse']),2)
df3['error'] = round(((df3['prediction'] - df3['actual'])),2)
df3['%error'] = (df3['error']/df3['actual'])*100
df3.columns =['sum of predictions', 'sum of actuals', 'mse', 'rmse', 'error', '%error']
print(df3)

            sum of predictions  sum of actuals            mse    rmse  \
2019-03-01       106937.082546          122566   61265.904195  247.52   
2019-04-01       100772.970137          109520   52831.173222  229.85   
2019-05-01        87969.038044           86028   27247.324894  165.07   
2019-06-01        67740.025922           64519   16040.442112  126.65   
2019-07-01        58971.390036           60027   12911.884787  113.63   
2019-08-01        57451.422003           55343   11900.757567  109.09   
2019-09-01        66313.465882           73711   19739.594451  140.50   
2019-10-01        86709.884309           91436   33557.450624  183.19   
2019-11-01       125842.942427          119083   53029.298064  230.28   
2019-12-01       148965.864814          151669   89977.743191  299.96   
2020-01-01       161636.381583          157278   98192.266490  313.36   
2020-02-01       146545.874211          152810   98592.117287  313.99   
2020-03-01       140737.689582          131657   59

In [9]:
df3.apply(Summary)

,sum of predictions,sum of actuals,mse,rmse,error,%error
min,57451.42,55343.00,11900.76,109.09,-16912.80,-17.35
max,168440.30,159230.00,108511.76,329.41,12731.61,10.53
mean,103758.41,104528.46,50508.66,212.10,-770.04,-0.78
sum,2490201.94,2508683.00,1212207.75,5090.37,-18481.08,-18.60
count,24.00,24.00,24.00,24.00,24.00,24.00


for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False, period=1)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

for m in meter_ids:
    df = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df.columns =['MeterNo', 'WeekNo', 'kWh']
    y = df.set_index(['WeekNo'])
    y = y.loc[y['MeterNo'] == m]
    y.drop(y.columns[[0]], axis = 1, inplace = True)
    #print(y)
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
                results = mod.fit()
                print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
            except: 
                continue

for m in meter_ids:
    df_meters = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, 
                            squeeze=True, dayfirst=True, engine='python')
    df_meters['Date'] = pd.to_datetime(df_meters['Date'], dayfirst=True)
    df_meters.columns =['MeterNo', 'Month', 'kWh']
    y = df_meters.set_index('Month')
    df_DDH = pd.read_csv('DDH_Monthly.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
    df_DDH['Date'] = pd.to_datetime(df_DDH['Date'], dayfirst=True)
    df_DDH.columns =['Month', 'DDH']
    z = df_DDH.set_index('Month')
    y = merge(y, z, left_index=True, right_index=True)
    y = y.loc[y['MeterNo'] == m]
    y.drop(y.columns[[0]], axis = 1, inplace = True)
    endog = y.iloc[:,0]
    exog = y.iloc[:,1:]
    freq = 'MS' # month start
    mod = sm.tsa.statespace.SARIMAX(endog = endog,
                                order=(1, 1, 1),
                                seasonal_order=(0, 1, 1, 12),
                                exog = exog,
                                freq = freq,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results = mod.fit()
    pred = results.get_prediction(start=pd.to_datetime(pred_date), dynamic=False)
    y_forecasted = pred.predicted_mean
    y_forecasted =y_forecasted.to_frame()
    y_truth = y[pred_date:]
    df1 = merge(y_forecasted, y_truth, left_index=True, right_index=True)
    df1.drop(df1.columns[[2]], axis = 1, inplace = True)
    df1.columns =['prediction', 'actual']
    df1['error'] = round(((df1['prediction'] - df1['actual'])),2)
    df1a = df1a.append(df1)


p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))